# Results
In this notebook we analyse the results from our experiments.

In [ ]:
nohup python3 main.py -s ydata -n 5 > experiment_n5_ydata.log 2>&1 &

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn import preprocessing
from matplotlib.lines import Line2D
from scipy.stats import mannwhitneyu

models = ['benchmark', 'dpctgan', 'ydata', 'ctgan', 'synthcity','datasynthesizer']
# note: nbsynthetic is not included due to failures. more info in our paper
datasets = ['breast', 'census', 'heart', 'student']
techniques = ['knn', 'lr', 'nn']

## Accuracy

In [13]:
for dataset in datasets:
    df = pd.read_csv(f'../measurements/accuracy_{dataset}.csv')
    
    # Compute the mean accuracy per method and round to 3 decimal places
    average_accuracy = df.groupby('name').mean().round(3)
    
    # Display the results
    print(f"Average accuracy for {dataset}:")
    print(average_accuracy)
    print("\n")

Average accuracy for breast:
                   knn     lr     nn
name                                
benchmark        0.739  0.685  0.696
ctgan            0.717  0.715  0.701
datasynthesizer  0.576  0.478  0.408
dpctgan          0.602  0.578  0.628
synthcity        0.750  0.772  0.758


Average accuracy for census:
                   knn     lr     nn
name                                
benchmark        0.802  0.846  0.840
ctgan            0.789  0.825  0.811
datasynthesizer  0.743  0.754  0.740
dpctgan          0.546  0.373  0.436
synthcity        0.697  0.713  0.559


Average accuracy for heart:
                   knn     lr     nn
name                                
benchmark        0.798  0.838  0.814
ctgan            0.543  0.417  0.614
datasynthesizer  0.455  0.455  0.467
dpctgan          0.482  0.475  0.475
synthcity        0.707  0.798  0.826


Average accuracy for student:
                   knn     lr     nn
name                                
benchmark        0.788  0.8

## Energy



In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn import preprocessing
from matplotlib.lines import Line2D
from scipy.stats import mannwhitneyu

models = ['benchmark', 'dpctgan', 'ydata', 'ctgan', 'synthcity', 'datasynthesizer']
techniques = ['knn', 'lr', 'nn']
datasets = ['breast', 'census', 'heart', 'student']

for dataset in datasets:
    df = pd.read_csv(f'../measurements/energy_{dataset}.csv')
    df.columns = ['label', 'timestamp', 'duration', 'pkg', 'dram', 'socket']
    
    # Convert to seconds and Joules
    df['duration'] *= 10**-6
    df['pkg'] *= 10**-6
    df['dram'] *= 10**-6

    # Extract method and technique from the label
    df['method'] = df['label'].apply(lambda x: x.split('_')[1] if '_' in x else 'benchmark')
    df['technique'] = df['label'].apply(lambda x: x.split('_')[0])  # Extract knn, lr, or nn

    # Compute total energy consumption (pkg + dram)
    df['total_energy'] = df['pkg'] + df['dram']

    # Iterate over methods and techniques to display results
    for method in models:
        for technique in techniques:
            subset = df[(df['method'] == method) & (df['technique'] == technique)]
            
            if subset.empty:
                continue  # Skip if no data is found for this method-technique combination
            
            # Compute averages and standard deviations
            energy_stats = subset.agg({
                'duration': ['mean', 'std'],
                'pkg': ['mean', 'std'],
                'dram': ['mean', 'std'],
                'total_energy': ['mean', 'std']
            }).round(6)  # Rounded for precision
            
            # Display results
            print(f"Energy consumption statistics for {dataset} - Method: {method} - ML technique: {technique}")
            print(f"  Average duration: {energy_stats.loc['mean', 'duration']} (σ: {energy_stats.loc['std', 'duration']}) seconds")
            print(f"  Average pkg: {energy_stats.loc['mean', 'pkg']} (σ: {energy_stats.loc['std', 'pkg']}) Joules")
            print(f"  Average dram: {energy_stats.loc['mean', 'dram']} (σ: {energy_stats.loc['std', 'dram']}) Joules")
            print(f"  Average total energy used: {energy_stats.loc['mean', 'total_energy']} (σ: {energy_stats.loc['std', 'total_energy']}) Joules")
            print("-" * 80)


Energy consumption statistics for breast - Method: benchmark - ML technique: knn
  Average duration: 0.017603 (σ: 0.001901) seconds
  Average pkg: 1.860432 (σ: 0.058292) Joules
  Average dram: 0.033789 (σ: 0.003927) Joules
  Average total energy used: 1.894221 (σ: 0.056764) Joules
--------------------------------------------------------------------------------
Energy consumption statistics for breast - Method: benchmark - ML technique: lr
  Average duration: 0.016098 (σ: 0.000126) seconds
  Average pkg: 3.12769 (σ: 0.131047) Joules
  Average dram: 0.03053 (σ: 0.001169) Joules
  Average total energy used: 3.15822 (σ: 0.132185) Joules
--------------------------------------------------------------------------------
Energy consumption statistics for breast - Method: benchmark - ML technique: nn
  Average duration: 0.952235 (σ: 0.147437) seconds
  Average pkg: 46.449734 (σ: 5.35669) Joules
  Average dram: 2.011579 (σ: 0.520407) Joules
  Average total energy used: 48.461314 (σ: 5.847273) Jou

## Anonymeter

In [ ]:
#TODO